In [5]:
using PerlaTonettiWaugh

# Steady-State Comparison

In [6]:
params = parameter_defaults()
baseline = merge(params, (d = params.d_0,))
baseline_sol = stationary_algebraic(baseline, settings_defaults());

In [7]:
new_params = merge(params, (d = params.d_T,))
new_sol = stationary_algebraic(new_params, settings_defaults());

In [8]:
@show (new_sol.g, baseline_sol.g);

(new_sol.g, baseline_sol.g) = (0.010250822794505147, 0.007913401963163525)


In [9]:
@show (1 - new_sol.λ_ii, 1 - baseline_sol.λ_ii);

(1 - new_sol.λ_ii, 1 - baseline_sol.λ_ii) = (0.14442864132703293, 0.10629127170507902)


In [10]:
@show (new_sol.U_bar, baseline_sol.U_bar);

(new_sol.U_bar, baseline_sol.U_bar) = (17.534340776670454, 12.322561075850931)


In [11]:
@show 100*consumption_equivalent(new_sol.U_bar, baseline_sol.U_bar, params);

100 * consumption_equivalent(new_sol.U_bar, baseline_sol.U_bar, params) = 11.181899506570137


# Transition Dynamics

In [12]:
using CSV, Plots, JSON, BenchmarkTools, Parameters, DataFrames, Suppressor
gr(fmt = :png)

Plots.GRBackend()

In [13]:
overall_parameters = parameter_defaults();

In [14]:
settings = settings_defaults(); # numerical solution settings
@unpack stationary_T, stationary_0, change_welfare, change_trade = compare_steady_states(overall_parameters, settings);

Note that everything from here on out is using the **numerical** solution, so the values might be slightly different than the algebraic one. 

We do this so that the endpoints are valid targets for the dynamical experiment. 

To use the algebraic one, simply add `algebraic = true` to the `compare_steady_states` function above. 

In [15]:
@show (stationary_T.g, stationary_T.z_hat, stationary_T.Ω, stationary_T.L_tilde); # inspect time T solution

(stationary_T.g, stationary_T.z_hat, stationary_T.Ω, stationary_T.L_tilde) = (0.01025616153886866, 1.8523062408835727, 0.6508139010452779, 0.24606267582507552)


In [16]:
@show (stationary_0.g, stationary_0.z_hat, stationary_0.Ω, stationary_0.L_tilde); # inspect time 0 solution

(stationary_0.g, stationary_0.z_hat, stationary_0.Ω, stationary_0.L_tilde) = (0.00792017047430068, 1.9881975410345563, 0.6852422606342697, 0.24226457898667145)


In [17]:
cachename = model_cachename(overall_parameters, settings) # uniquely identify the run by parameters + settings
cachepath = joinpath(pwd(), "data", cachename * ".csv")
use_cache = true # change this to false if you don't want to use the cache for a run
write_cache = true # change this to false if you don't want to generate *any* cache files for a run

true

In [18]:
cachename # this is the hash for this run

"2357024147638739296"

In [ ]:
if isfile(cachepath) && use_cache # read if possible
    println("using cache.")
    solved = CSV.read(cachepath);
else
    println("solving without cache.")
    @time solution = @suppress solve_transition(overall_parameters, settings);
    solved = solution.results 
end

In [ ]:
if write_cache
    mkpath(joinpath(pwd(), "data")); # if this doesn't eist for whatever reason
    CSV.write(cachepath, solved); # write DataFrame
    write(joinpath(pwd(), "data", cachename * ".json"), JSON.json((parameters = overall_parameters, settings = merge(settings, (interp = typeof(settings.interp), stationary_x0 = typeof(settings.stationary_x0), fixedpoint_x0 = typeof(settings.fixedpoint_x0) ))))); # write parameters
else
    # nothing
end

## Welfare

Here's welfare inclusive of the transition path:

In [35]:
ce_transition = 100*consumption_equivalent(solved.U[1], stationary_0.U_bar, parameter_defaults());
@show ce_transition;

ce_transition = 10.802117356508889


Plots for the transition path can be found in the notebook [TransitionDynamicsFigures.ipynb](https://github.com/jlperla/PerlaTonettiWaugh.jl/blob/master/TransitionDynamicsFigures.ipynb)